In [2]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
import requests

In [15]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
driver.get(urls[0]['url'])
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

Mozilla/5.0 (X11; CrOS i686 3912.101.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36


KeyError: 0

In [2]:
url  = 'https://www.amazon.sa/s?k=Beauty+land+gardens&ref=bl_dp_s_web_0'

In [ ]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
driver.get(url)
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

In [19]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

In [3]:
def return_ele(name, soup):
    try:
        return soup.find('span', text=re.compile(name)).next_element.next_element.next_element.next_element.text.strip()
    except:
        return ''
    

In [89]:
free_colors = return_ele('اللون', soup)
raw_materials = return_ele('المادة', soup)
brand = return_ele('العلامة التجارية', soup)
free_colors, raw_materials, brand

('كوفي', 'فايبر', 'بيوتي لاند جاردنز')

In [ ]:
def return_ele(name, soup):
    try:
        return soup.find('span', text=re.compile(name)).next_element.next_element.next_element.next_element.text.strip()
    except:
        return ''
    



urls = pd.read_excel('amz_update_url.xlsx')
list_urls = []

for indx, row in urls.iterrows():
    list_urls.append(
        {'url': row['url'], 'cat1': row['cat1'], 'cat2': row['cat2'], 'cat3': row['cat3']},
    )

df = pd.read_excel('amz_product_model.xlsx')

for i, url1 in enumerate(list_urls):
    print('Count: ', i)
    url = url1['url']
    print('URL: ', url)
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    cookies = {'session': '134-8225175-0355220'+ str(i)}
    time.sleep(1)
    r = requests.get(url, headers=headers, cookies=cookies)
    time.sleep(3)
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        name = soup.find('span', {'id': 'productTitle'}).text.strip()
    except:
        headers = {'User-Agent': userAgent}
        cookies = {'session': '134-8225175-0355220'+ str(i)}
        time.sleep(3)
        r = requests.get(url, headers=headers, cookies=cookies)
        time.sleep(1)
        soup = BeautifulSoup(r.text, "html.parser")
        name = soup.find('span', {'id': 'productTitle'}).text.strip()

    try:
        price1 = soup.find('span', {'class': 'a-price-whole'}).text.strip().split('.')[0]
    except:
        continue
    price2 = soup.find('span', {'class': 'a-price-fraction'}).text.strip()
    price = price1 + '.'+  price2
    free_colors = return_ele('اللون', soup)
    raw_materials = return_ele('المادة', soup)
    brand = return_ele('العلامة التجارية', soup)
    style = return_ele('النمط', soup)
    included_ingredients = return_ele('المكونات المشمولة', soup)
    free_colors1 = return_ele('نوع اللمسة النهائية', soup)
    try:
        product_details = soup.find('table', {'id': 'productDetails_techSpec_section_1'}).text.strip()
    except:
        product_details = ''

    is_in_stock = soup.find('div', {'id': 'availability'}).find('span').text.strip()

    try:
        description = soup.find('div', {'id': 'featurebullets_feature_div'}).text.strip()
    except AttributeError:
        description = ''

    try:
        images = soup.find('div', {'id': 'altImages'}).find_all('img', )
        len(images)
    except AttributeError:
        continue

    list_images = []
    for img in images[:-1]:
        # print(img['src'].split('_'))
        list_images.append(''.join([img['src'].split('_')[0], img['src'].split('_')[-1].replace('.', '')] ))

    base_image = list_images[0]
    add_images = ','.join(list_images[1:])
    data = {
        'name': name,
        'link_url': url,
        'price': price,
        'free_colors': free_colors,
        'raw_materials': raw_materials,
        'brand': brand,
        'style': style,
        'included_ingredients': included_ingredients,
        'free_colors1': free_colors1,
        'product_details': product_details,
        'is_in_stock': is_in_stock,
        'description': description,
        'base_image': base_image,
        'add_images': add_images,
    }
    
    df1 = pd.DataFrame([data])
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('amz_update_products01.xlsx')

In [12]:
df = None
df1 = pd.read_excel('amz_update_products02.xlsx')
df2 = pd.read_excel('amz_update_products01.xlsx')

In [13]:
df =  pd.concat([df1, df2], ignore_index=True)

In [14]:
df.to_excel('amz_update_pro2.xlsx')

In [29]:
def extract(name, soup):
    try:
        return soup.find('th', text=re.compile(name)).next_element.next_element.next_element.text.strip()
    except:
        return ''

In [33]:
spec = extract('الخصائص', soup)
products_size = extract('أبعاد الشحنة', soup)
included_ingredients = extract('المكونات المشمولة', soup)

spec, products_size, included_ingredients

('\u200eSelf Watering',
 '\u200e44 x 24 x 24 سم; 3.85 كيلو غرام',
 '\u200eمستشعر مياه، أنبوب قياس الماء')

In [22]:
soup.find('div', {'id': 'prodDetails'})

<div class="a-section" id="prodDetails"> <h2>
                          معلومات المنتج </h2>
<div class="a-row a-spacing-top-base"> <div class="a-column a-span6"> <div class="a-row a-spacing-base"> <div class="a-row a-expander-container a-expander-extend-container"> <div class="a-row"> <div class="a-column a-span6"> <h1 class="a-size-medium a-spacing-small">التفاصيل التقنية</h1> </div> </div> <div aria-expanded="true" class="a-expander-content a-expander-extend-content"> <div class="a-row a-expander-container a-expander-inline-container"> <div aria-expanded="true" class="a-expander-content a-expander-section-content a-section-expander-inner"> <table class="a-keyvalue prodDetTable" id="productDetails_techSpec_section_1" role="presentation"> <tbody><tr> <th class="a-color-secondary a-size-base prodDetSectionEntry"> اللون </th> <td class="a-size-base prodDetAttrValue">
                ‎أسود </td> </tr> <tr> <th class="a-color-secondary a-size-base prodDetSectionEntry"> الحجم </th> <td cla